In [1]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import onlypores as op
from pathlib import Path

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [ ]:
database.columns

In [5]:
ids = database['Id']

paths = database['Frontal 90 Right']

onlypores_paths = database['Onlypores']

masks = database['Material Mask']

segmenteds = database['Binary']

In [6]:
onlypores_paths = [None,None]

In [8]:
for i in range(len(ids)):

    if (paths[i] != None) and ((onlypores_paths[i] is None) or (masks[i] is None) or (segmenteds[i] is None)):    

        print(f'Processing {ids[i]}')

        try:

            print(f'Processing {paths[i]}')

            xct_folder = Path(paths[i])
            # volume = read_sequence(xct_folder)
            volume = tifffile.imread(xct_folder)
            from time import time
            start = time()
            onlypores, sample_mask, binary = op.onlypores(volume)
            print(time()-start)
            start = time()
            onlypores2, sample_mask2, binary2 = op.onlypores_parallel(volume)
            print(time()-start)

            #check if the results are the same
            if not np.all(onlypores == onlypores2):
                print('onlypores are different')
                break

            break

            output_folder = xct_folder.parent / 'segmentation'

            #save the segmented volume
            write_sequence(output_folder, 'onlypores', onlypores.astype(np.uint8)*255)
            #save the sample mask
            write_sequence(output_folder, 'sample_mask', sample_mask.astype(np.uint8)*255)
            #save the segmented volume
            write_sequence(output_folder, 'binary', binary.astype(np.uint8)*255)

            onlypores_paths[i] = str(output_folder / 'onlypores')
            masks[i] = str(output_folder / 'sample_mask')
            segmenteds[i] = str(output_folder / 'binary')
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

Processing 1_26
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\26\frontal_90right.tif
masking
computing otsu
thresholding with value:  108
137.1868977546692
masking
computing otsu
thresholding with value:  108
125.04253196716309


In [7]:
# database['Onlypores'] = onlypores_paths
# database['Material Mask'] = masks
# database['Binary'] = segmenteds

In [8]:
# database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)